# 1 Introducción

El siguiente ejemplo combina 2 imagenes parametrisadas, en una unica foto. La combinacion se realiza colocando la mitad de cada imagen en la imagen resultado. Esta combinacion puede ser vertical como horizontal, dependiendo de un parametro que se puede cambiar. Nota: las imagenes deben ser de igual tamaño.

El objetivo es calcular el tiempo que toma esta combinacion en forma paralela usando GPU con CUDA para observar la diferencia de tiempo con el modo secuencial.

---
# 2 Armado del ambiente
Toma la dirección web de 2 imagenes con acceso público en internet, la deja disponible al contexto de ejecución del cuaderno colab.

In [2]:
#@title ## 2.1 Parámetros de ejecución
#@markdown ---
#@markdown ### Especifique la URL de la imagen:
url_imagen = "https://github.com/bustamanteandres/BustamanteSOA/blob/main/HPC/cara2.jpg?raw=true" #@param {type:"string"}
url_imagen2 = "https://github.com/bustamanteandres/BustamanteSOA/blob/main/HPC/cara4.jpg?raw=true" #@param {type:"string"}
#@markdown ---

!wget {url_imagen} -O imagen.jpg
!wget {url_imagen2} -O imagen2.jpg



--2020-11-22 23:57:05--  https://github.com/bustamanteandres/BustamanteSOA/blob/main/HPC/cara2.jpg?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/bustamanteandres/BustamanteSOA/raw/main/HPC/cara2.jpg [following]
--2020-11-22 23:57:05--  https://github.com/bustamanteandres/BustamanteSOA/raw/main/HPC/cara2.jpg
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bustamanteandres/BustamanteSOA/main/HPC/cara2.jpg [following]
--2020-11-22 23:57:06--  https://raw.githubusercontent.com/bustamanteandres/BustamanteSOA/main/HPC/cara2.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connecte

---
## 2.2 Instala en el cuaderno el módulo CUDA de Python.

In [4]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 21.5MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=d76714a5528a562e266c883be9cd1e900d495efc41c377ee3f21f20a4f7196dc
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=fee50cf13ea37ebc3a13fc98effabd2f75340e041cf94f9f87864f73819d9bfb
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


---
# 3 Desarrollo
Ejecución del algoritmo escala de grises en GPU.

In [3]:
%matplotlib inline

try:
  from datetime import datetime
  import matplotlib.pyplot as plt
  import numpy
  from PIL import Image 
  import pycuda.driver as cuda
  import pycuda.autoinit
  from pycuda.compiler import SourceModule

  # --------------------------------------------
  # Definición de función que transforma el tiempo en  milisegundos 
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
  
  #@markdown ---
  #@markdown ### Seleccione el modo de combinacion de imagen
  combinacion_vertical = True #@param {type:"boolean"}
  #@markdown ---

  tiempo_inicio_total = datetime.now()
  # Cargo las imagenes guardadas en el "Armado del ambiente"
  img_nombre = 'imagen.jpg'
  img2_nombre = 'imagen2.jpg'
  image = Image.open(img_nombre) 
  image2 = Image.open(img2_nombre)

  # Obtengo dimensiones de ambas imagenes
  img_ancho, img_alto = image.size
  img2_ancho, img2_alto = image2.size
  print("Imagen 1: " + img_nombre + " -" + image.mode + "- [" + str(img_ancho) + ", " + str(img_alto ) + "]" )
  print("Imagen 2: " + img2_nombre + " -" + image2.mode + "- [" + str(img2_ancho) + ", " + str(img2_alto ) + "]" )  
  if image.size != image2.size:
    raise Exception("Las imagenes no son del mismo tamaño")
  
  # Convierto la imagen comprimida en JPEG/PNG a array
  img1_cpu = numpy.asarray(image)
  img2_cpu = numpy.asarray(image2)

  # Genero el array resultado (3 pixeles usa RGB * (el tamaño del array))
  if img1_cpu.size > img2_cpu.size:
    img_result_cpu = numpy.empty_like(img1_cpu)
  else:
    img_result_cpu = numpy.empty_like(img2_cpu)

  # Reservo los 3 vectores en GPU(3 pixeles usa RGB * (el tamaño del array))
  img1_gpu = cuda.mem_alloc(img1_cpu.nbytes)
  img2_gpu = cuda.mem_alloc(img2_cpu.nbytes)
  img_result_gpu = cuda.mem_alloc(img_result_cpu.nbytes)

  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod( img1_gpu, img1_cpu )
  cuda.memcpy_htod( img2_gpu, img2_cpu )
  cuda.memcpy_htod( img_result_gpu, img_result_cpu)

  # CPU - Defino las funciones kernel que ejecutará en GPU.
  module = SourceModule("""
  __global__ void kernel_img_vertical( int ancho, int alto, char *img1, char *img2, char *img_R)
  {
    // Calculo las coordenadas del Thread en dos dimensiones.
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    int idy = threadIdx.y + blockIdx.y*blockDim.y;
    int mitad_alto = alto / 2;
    // Verifico que los Thread, esten dentro de las dimensiones de la imagen.
    if( idx < ancho && idy < alto )
    {
      // Escribo el color del pixel.
      if(idy < mitad_alto){
        img_R[((idx + (idy * ancho)) * 3)] = img1[((idx + (idy * ancho)) * 3)];
        img_R[((idx + (idy * ancho)) * 3) + 1] = img1[((idx + (idy * ancho)) * 3) + 1];
        img_R[((idx + (idy * ancho)) * 3) + 2] = img1[((idx + (idy * ancho)) * 3) + 2];
      }else{
        img_R[((idx + (idy * ancho)) * 3)] = img2[((idx + (idy * ancho)) * 3)];
        img_R[((idx + (idy * ancho)) * 3) + 1] = img2[((idx + (idy * ancho)) * 3) + 1];
        img_R[((idx + (idy * ancho)) * 3) + 2] = img2[((idx + (idy * ancho)) * 3) + 2];
      }
    }
  }

  __global__ void kernel_img_horizontal( int ancho, int alto, char *img1, char *img2, char *img_R)
  {
    // Calculo las coordenadas del Thread en dos dimensiones.
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    int idy = threadIdx.y + blockIdx.y*blockDim.y;
    int mitad_ancho = (int)(ancho / 2);
    // Verifico que los Thread, esten dentro de las dimensiones de la imagen.
    if( idx < ancho && idy < alto )
    {
      // Escribo el color del pixel.
      if(idx < mitad_ancho){
        img_R[((idx + (idy * ancho)) * 3)] = img1[((idx + (idy * ancho)) * 3)];
        img_R[((idx + (idy * ancho)) * 3) + 1] = img1[((idx + (idy * ancho)) * 3) + 1];
        img_R[((idx + (idy * ancho)) * 3) + 2] = img1[((idx + (idy * ancho)) * 3) + 2];
      }else{
        img_R[((idx + (idy * ancho)) * 3)] = img2[((idx + (idy * ancho)) * 3)];
        img_R[((idx + (idy * ancho)) * 3) + 1] = img2[((idx + (idy * ancho)) * 3) + 1];
        img_R[((idx + (idy * ancho)) * 3) + 2] = img2[((idx + (idy * ancho)) * 3) + 2];
      }
    }
  }
  """)

  # CPU - Genero la función kernel.
  if combinacion_vertical:
    kernel = module.get_function("kernel_img_vertical")
  else:
    kernel = module.get_function("kernel_img_horizontal")
  tiempo_ini_gpu = datetime.now()

  dim_hilo_x = 16
  dim_bloque_x = numpy.int((img_ancho + dim_hilo_x-1) / dim_hilo_x)
  dim_hilo_y = 16
  dim_bloque_y = numpy.int((img_alto+dim_hilo_y-1) / dim_hilo_y)

  print("Thread: [", dim_hilo_x, ",", dim_hilo_y, " ], Bloque : [", dim_bloque_x, ",", dim_bloque_y, "]" )
  print("Total de Thread: [", dim_hilo_x * dim_bloque_x, ",", dim_hilo_y * dim_bloque_y, " ]", " = ", dim_hilo_x*  dim_bloque_x * dim_hilo_y * dim_bloque_y )
  tiempo_inicio_gpu = datetime.now()

  kernel(numpy.int32(img_ancho), numpy.int32(img_alto), img1_gpu, img2_gpu, img_result_gpu, block=( dim_hilo_x, dim_hilo_y, 1 ), grid=(dim_bloque_x, dim_bloque_y,1) )

  tiempo_total_gpu = datetime.now() - tiempo_inicio_gpu

  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh(img_result_cpu, img_result_gpu)

  # Muestro las imagenes originales..
  plt.figure()
  imgplot = plt.imshow(img1_cpu)
  plt.figure()
  imgplot = plt.imshow(img2_cpu)

  # Muestro la imagen luego de aplicar la combinacion.
  plt.figure()
  imgplot = plt.imshow(img_result_cpu)

  tiempo_total = datetime.now() - tiempo_inicio_total
  print("Tiempo total:", tiempo_en_ms(tiempo_total), "ms")
  print("Tiempo GPU:", tiempo_en_ms(tiempo_total_gpu), "ms")
except Exception as excep:
  print("Error: ", excep)

Error:  No module named 'pycuda'


---
# 4 Tabla de pasos


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura de las direcciones URL de las imagenes (jpg) a procesar.
CPU      |  wget url_imagen       | Lectura de las direcciones URL de la imagenes (jpg) a procesar.
CPU      | pip install pycuda    | Instala en el cuaderno los driver de CUDA para Python.
CPU      |  matplotlib inline    | Macro de Colab para mostrar imagenes.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  tiempo_en_ms          | Define una funcion para obtener el tiempo en milisegundos a partir de una fecha.
CPU      |  @param                | Lectura del modo de combinacion de imagen.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  Image.open()          | Abre los archivos de las imagenes.
CPU      |  numpy.asarray(imagen) | Convierte el formato comprimido JPG a RAW.
CPU      |  numpy.empty_like(()   | Genera el array destino, que tendrá a la imagen resultado.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria para las imagenes en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copio los valores de las imagenes al GPU.
CPU      |  SourceModule()        | Posee el còdigo del kernel.
CPU      |  module.get_function() | Convierte el texto del kernel en funcion de Python.
CPU      |  dim_hilo_x, dim_hilo_y| Calcula las dimensiones para la ejecuciòn de 2D.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU, enviando los parametros.
CPU      |  print()               | Informa los atributos de la imagenes.
CPU      | cuda.memcpy_dtoh()     | Copia desde la memoria GPU al CPU.
CPU      |  plt.imshow            | Muestra la imagenes originales.
CPU      |  plt.imshow            | Muestra la imagen resultado.
CPU      |  print()          | Muestra el tiempo total y el tiempo que tomo la funcion GPU.

---
# 5 Conclusiones

Como conclusion podemos observar que al realizar la combinacion en paralelo de las 2 imagenes, usando threads del GPU, dicha combinacion se realiza mucho mas rapido que si se hiciera secuencialmente. Si bien la inicializacion toma bastante tiempo, al tratarse de una combinacion de imagenes, el procesamiento siempre sera grande, por lo que termina siendo mas eficiente.

---
# 6 Bibliografía

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)
